In [1]:
from __future__ import division
import pdb 
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
import util
import seaborn as sns 
import clean_data as cd
import benchmark
import group_models

In [2]:
pd2007 = cd.import_data(2007)
pd2012 = cd.import_data(2012)

all_data = pd.concat([pd2007, pd2012])
prepped_data = cd.prep_for_classification(all_data)
X, y = cd.feature_target_split(prepped_data)

/Library/Python/2.7/site-packages/IPython/core/interactiveshell.py:3006: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [3]:
# grades are still here
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
train_payouts = all_data.ix[X_train.index]['payout_prop']
test_payouts = all_data.ix[X_test.index]['payout_prop']

print "Benchmark for all data on A/B/C: %.2f" %benchmark.total_return(all_data, filter_col='grade', filter_vals=['A', 'B', 'C'])
print "Benchmar for all data on all loans: %.2f" %benchmark.total_return(all_data)
print "Benchmark for all data is : %.2f" %np.mean(all_data['payout_prop'])
print "Benchmark for 2012 data is: %.2f" %np.mean(pd2012['payout_prop'])
print "Benchmark for 2007 data is: %.2f" %np.mean(pd2007['payout_prop'])

lr_params = { 
    'C': 100,
    'class_weight': {False: 15},
    'penalty': 'l2',
    'n_jobs': -1
}

models = group_models.fit(X_train, y_train, 'grade', LogisticRegression, lr_params)
group_models.group_results(X_test, y_test, test_payouts, 'grade', models)

Benchmark for all data on A/B/C: 1.08
Benchmar for all data on all loans: 1.08
Benchmark for all data is : 1.08
Benchmark for 2012 data is: 1.09
Benchmark for 2007 data is: 1.08
Making group for group A
Making group for group B
Making group for group C
Making group for group D
Making group for group E
Making group for group F
Making group for group G
Precision for group A: 0.99
Benchmark for group A: 1.09
[[5999   36]
 [  36  345]]
Precision for group B: 0.99
Benchmark for group B: 1.16
[[9318  362]
 [ 121 1244]]
Precision for group C: 0.77
Benchmark for group C: 0.99
[[  17 5594]
 [   5 1212]]
Precision for group D: 1.00
Benchmark for group D: 1.15
[[   1 2855]
 [   0  848]]
Precision for group E: 0.60
Benchmark for group E: 0.58
[[  3 582]
 [  2 187]]
Precision for group F: nan
Benchmark for group F: nan
[[ 0 79]
 [ 0 41]]
Precision for group G: nan
Benchmark for group G: nan
[[ 0 11]
 [ 0  4]]
Overall precision: 15338 164
Overall benchmark: 1.13


/Library/Python/2.7/site-packages/numpy/core/fromnumeric.py:2652: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


In [4]:
all_lr_model = LogisticRegression(**lr_params)
all_lr_model.fit(X_train.drop('grade', axis=1), y_train)

LogisticRegression(C=100, class_weight={False: 15}, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=-1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [5]:
predicted = all_lr_model.predict(X_test.drop('grade', axis=1))
cm_all_model = confusion_matrix(y_test, predicted, labels=[True, False])

In [6]:
precision_all = cm_all_model[0,0] / np.sum(cm_all_model[:, 0])
print precision_all

0.988283339005


In [7]:
all_model_indices = [X_test.index[i] for i in range(len(X_test)) if predicted[i] == 1]
predicted_df = all_data.ix[all_model_indices]
benchmark.total_return(predicted_df)

1.1582344506470594

In [8]:
print cm_all_model

[[20328  4529]
 [  241  3804]]


In [9]:
test_copy = X_test.copy()
test_copy['prediction'] = predicted
grouped_test = test_copy.groupby('grade')

for gname, df in grouped_test:
    y_group = y_test.ix[df.index]
    print (" Total pred 1: ", np.sum(df['prediction']))
    cm_group = confusion_matrix(y_group, df['prediction'], labels=[True, False])
    print "Group %s" %gname
    print cm_group
    print "Precision: %.2f" %(float(cm_group[0, 0]) / float(cm_group[0,0] + cm_group[1, 0]))
    print("=======================================================")
    # grab rows by grade
    

(' Total pred 1: ', 6063)
Group A
[[6001   34]
 [  62  319]]
Precision: 0.99
(' Total pred 1: ', 9055)
Group B
[[8938  742]
 [ 117 1248]]
Precision: 0.99
(' Total pred 1: ', 3986)
Group C
[[3936 1675]
 [  50 1167]]
Precision: 0.99
(' Total pred 1: ', 1298)
Group D
[[1287 1569]
 [  11  837]]
Precision: 0.99
(' Total pred 1: ', 150)
Group E
[[149 436]
 [  1 188]]
Precision: 0.99
(' Total pred 1: ', 16)
Group F
[[16 63]
 [ 0 41]]
Precision: 1.00
(' Total pred 1: ', 1)
Group G
[[ 1 10]
 [ 0  4]]
Precision: 1.00
